# DPH algorithm implementation

Dominance Principle was originally proposed for Multi Demand Knapsack in

> "A new polynomial time algorithm for 0–1 multiple knapsack problem based
        on dominant principles", 
        
by Balachandar and Kannan.
        
Dominance Principle was adapted to MDMKP in

> "A new heuristic approach for Knapsack/Covering Problem",

also by Balachandar and Kannan. This method is what this notebook implements. 

---

First, we must include my code for metaheuristics for the MDMKP. This contains things like the problem definition and functions to score solutions, as well as the metaheuristics. 

In [46]:
include("compose.jl");

Now we are ready to implement the Dominance Principle Heuristic. We will do this step by step, so we can observe how each step changes the data. The first step involves sorting the problem demand constraint columns by totals of the columns, and going through this list, adding the sorted column's respective items into the knapsack, as long as at least one demand constraint is violated. 

In [32]:
"""Returns a bitlist of all zeros with the same length as the amount of problem dimensions. """
function generate_initial_bitarray(problem::Problem)
    initial_bitlist::BitArray = falses(length(problem.objective))
    
    demand_columns = [coeffs for (coeffs, bound) in problem.lower_bounds] #get list of demand constraint coefficients
    #convert list into matrix:
    demand_columns = permutedims(reshape(hcat(demand_columns...), (length(demand_columns[1]), length(demand_columns))))
    
    demand_bounds = [bound for (coeffs, bound) in problem.lower_bounds]
    demand_totals = [sum(demand_columns[:,i]) for i in 1:size(demand_columns)[2]]

    #we need to sort column_totals
    zipped::Vector{Tuple{Int,Int}} = []
    index = 0
    for total in demand_totals
        index += 1
        push!(zipped, tuple(total, index))
    end
    sort!(zipped, by=x->x[1], rev=true)
    
    #we can now loop over the columns in descending order
    for (column_total, column_index) in zipped
        
        #enable the bit
        initial_bitlist[column_index] = 1
        
        #decrease bounds by value of respective coefficients that were just enabled
        column_coefficients = demand_columns[:,column_index]
        for i in 1:length(column_coefficients)
            demand_bounds[i] -= column_coefficients[i]
        end
        
        #check that we should continue
        cont = false
        for bound in demand_bounds
            if bound > 0
                cont = true
                break
            end
        end
        if !cont
            break
        end
    end
    
    #we now have our initial starting bitlist
    return initial_bitlist
end

generate_initial_bitarray

Let's test this function:

In [36]:
problem = parse_file("./benchmark_problems/mdmkp_ct1.txt")[3]
initial_solution = generate_initial_bitarray(problem)

100-element BitArray{1}:
  true
 false
 false
  true
 false
 false
 false
 false
  true
  true
 false
 false
 false
     ⋮
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
  true

In [43]:
satisfies_demands(initial_solution, problem) #lets check that the demand constraints are satisfied

true

In [44]:
#are the dimension constraints?
satisfies_dimensions(initial_solution,problem)

true

In [48]:
#for how many problems will this heuristic generate a feasible solution?
total_valid = 0
for dataset in 1:9
    problems = parse_file("./benchmark_problems/mdmkp_ct$(dataset).txt")
    for problem in problems
        ini_sol = generate_initial_bitarray(problem)
        if is_valid(ini_sol, problem)
            total_valid += 1
        end
    end
end
total_valid

616

In [ ]:
#now we must construct the dimension intercept matrix
function generate_intercept_matrix(problem::Problem)
    